In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score
import matplotlib.pyplot as plt
def plot_roc_cur(fper, tper, title):  
    plt.plot(fper, tper, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC-' + title)
    plt.legend()
    # plt.savefig(plots_folder + 'roc.png')
    plt.show()
    
# label and label_pred both numpy arrays
def evaluate(label_pred, label, label_cols=range(11)):
    thresh = 0.5
    label_true = label
    
    # normalize so that pred_0 + pred_1 = 1
    for i in range(label_true.shape[1]//2):
        label_pred[:,2*i:2*i+2] = tf.nn.softmax(label_pred[:,2*i:2*i+2],axis=1)
        
    prob = label_pred*label_true
    
    print(': the overall accuracy is:', sum(sum(prob>thresh))/sum(sum(label>0)))
    print(': the overall recall is:', sum(sum(prob>thresh)[1::2])/sum(sum(label>0)[1::2]))
    
    meas_steps = label_cols
    result = pd.DataFrame(index=meas_steps, columns = ['tp','tn','fn','fp','tpr','fpr','min_dis', 'auc'])
    for i in range(len(meas_steps)):
        neg = prob[:,2*i]
        pos = prob[:,2*i+1]

        pos = pos[pos!=0]
        neg = neg[neg!=0]

        tp = sum(pos>thresh)
        fn = sum(pos<thresh)
        tn = sum(neg>thresh)
        fp = sum(neg<thresh)
        tpr = tp/(tp+fn+1e-9)
        fpr = fp/(fp+tn+1e-9)

        y_prob = np.append(pos, neg)
        y_true = np.append([1]*len(pos), [0]*len(neg))
        if len(pos) and len(neg):
            fper, tper, thresholds = roc_curve(np.append([1]*len(pos), [0]*len(neg)),np.append(pos, 1-neg))
            auc = roc_auc_score(np.append([1]*len(pos), [0]*len(neg)),np.append(pos, 1-neg))
            min_dis = np.sqrt(fper**2 + (1-tper)**2).min()
        else:
            min_dis = None
            auc = None
        result.iloc[i] = [tp,tn,fn,fp,tpr,fpr,min_dis,auc]
        
        plot_roc_cur(fper, tper, str(meas_steps[i]))
    
    total_sample = sum(sum(result[['tp', 'tn', 'fp', 'fn']].values))
    # final score
    result['score'] = (result.tp + result.tn + result.fp + result.fn)*result.auc/total_sample
    display(result)
    print(sum(result.score))

In [ ]:
evaluate(label_pred, label)